# Model Controller Tutorial: Roberta model (Multi Head)

> This notebook contains some example of how to use the Roberta-based models in this NLP library

- skip_showdoc: true
- skip_exec: true

In this series, we walk through some of the capability of this library: single-head classification, multi-head classification, multi-label classification, and regression. If you want a more detailed tutorial, check [this](https://anhquan0412.github.io/that-nlp-library/model_classification_tutorial.html) out

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import os

In [ ]:
#This will specify a (or a list) of GPUs for training
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [ ]:
from that_nlp_library.text_transformation import *
from that_nlp_library.text_augmentation import *
from that_nlp_library.text_main import *
from that_nlp_library.utils import seed_everything

In [ ]:
from underthesea import text_normalize
from functools import partial
from pathlib import Path
import pandas as pd
import numpy as np
import nlpaug.augmenter.char as nac
from datasets import load_dataset
import random
from transformers import RobertaTokenizer
from datasets import Dataset

# Define the custom augmentation function

In [ ]:
def nlp_aug_stochastic(x,aug=None,p=0.5):
    if not isinstance(x,list): 
        if random.random()<p: return aug.augment(x)[0]
        return x
    news=[]
    originals=[]
    for _x in x:
        if random.random()<p: news.append(_x)
        else: originals.append(_x)
    # only perform augmentation when needed
    if len(news): news = aug.augment(news)
    return news+originals

In [ ]:
aug = nac.KeyboardAug(aug_char_max=3,aug_char_p=0.1,aug_word_p=0.07)
nearby_aug_func = partial(nlp_aug_stochastic,aug=aug,p=0.3)

# Create a TextDataController object

We will reuse the data and the preprocessings in [this tutorial](https://anhquan0412.github.io/that-nlp-library/text_main.html) 

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')


In [ ]:
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names=['Division Name','Department Name'],
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         metadatas=['Title'],
                         content_transformations=[text_normalize,str.lower],
                         content_augmentations= [nearby_aug_func,str.lower], 
                         val_ratio=0.2,
                         batch_size=1000,
                         seed=42,
                         num_proc=20,
                         verbose=False
                        )

Define our tokenizer for Roberta

In [ ]:
_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Process and tokenize our dataset

In [ ]:
tdc.process_and_tokenize(_tokenizer,max_length=100,shuffle_trn=True)

Filter (num_proc=20):   0%|          | 0/18102 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/18101 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/18101 [00:00<?, ? examples/s]

Flattening the indices (num_proc=20):   0%|          | 0/18101 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/18101 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4526 [00:00<?, ? examples/s]

In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 18101
    })
    validation: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

# Model Experiment: Roberta Multi-Head Classification (with Hidden Layer Concatenation)

In [ ]:
from that_nlp_library.models.roberta.classifiers import *
from that_nlp_library.model_main import *
from sklearn.metrics import f1_score, accuracy_score

comet_ml is installed but `COMET_API_KEY` is not set.


## Define and train a custom Roberta model

In [ ]:
from transformers.models.roberta.modeling_roberta import RobertaModel

In [ ]:
num_classes = [len(tdc.label_lists[0]),len(tdc.label_lists[1])] 
num_classes

[3, 6]

In [ ]:
roberta_body = RobertaModel.from_pretrained('roberta-base')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# our model is more complex, so it's best to define some of its arguments
_model_kwargs={
    # overall model hyperparams
    'head_class_sizes':num_classes,
    'head_class': ConcatHeadSimple,
    'is_multilabel':tdc.is_multilabel, # False
    'is_multihead':tdc.is_multihead, # True
    'head_weights':[1,2], # weights for label 1 and label 2 This means L2's weight is twice as much as L1's
    # classfication head hyperparams
    'layer2concat':2, # you can change the number of layers to concat (default is 4, based on the paper)
    'classifier_dropout':0.1 
}

model = model_init_classification(model_class = RobertaHiddenStateConcatForSequenceClassification,
                                  cpoint_path = 'roberta-base', 
                                  output_hidden_states=True, # since we are using 'hidden layer contatenation' technique
                                  seed=42,
                                  body_model=roberta_body,
                                  model_kwargs = _model_kwargs)

metric_funcs = [partial(f1_score,average='macro'),accuracy_score]
controller = ModelController(model,tdc,seed=42)

Loading body weights. This assumes the body is the very first block of your custom architecture


And we can start training our model

In [ ]:
seed_everything(42)

In [ ]:
lr = 1e-4
bs=32
wd=0.01
epochs= 3

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics_classification,
              )

/home/quan/anaconda3/envs/fastai_v2/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score Division name,Accuracy Score Division name,F1 Score Department name,Accuracy Score Department name
1,No log,1.616623,0.416984,0.614229,0.663449,0.870968
2,1.893500,1.539352,0.525900,0.578878,0.668691,0.877817
3,1.893500,1.502877,0.451556,0.616217,0.684416,0.884887


In [ ]:
controller.trainer.model.save_pretrained('./sample_weights/my_model1')

## Make predictions

### Load trained model

In [ ]:
_model_kwargs

{'head_class_sizes': [3, 6],
 'head_class': that_nlp_library.models.roberta.classifiers.ConcatHeadSimple,
 'is_multilabel': False,
 'is_multihead': True,
 'head_weights': [1, 2],
 'layer2concat': 2,
 'classifier_dropout': 0.1}

In [ ]:
trained_model = model_init_classification(model_class = RobertaHiddenStateConcatForSequenceClassification,
                                          cpoint_path = Path('./sample_weights/my_model1'), 
                                          output_hidden_states=True,
                                          seed=42,
                                          model_kwargs = _model_kwargs)

controller = ModelController(trained_model,tdc,seed=42)

Some weights of the model checkpoint at sample_weights/my_model1 were not used when initializing RobertaHiddenStateConcatForSequenceClassification: ['body_model.pooler.dense.bias', 'body_model.pooler.dense.weight']
- This IS expected if you are initializing RobertaHiddenStateConcatForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaHiddenStateConcatForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Predict Train/Validation set

In [ ]:
df_val = controller.predict_ddict_classification(ds_type='validation')

-------------------- Start making predictions --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

In [ ]:
df_val = df_val.to_pandas()
df_val.head()

,Title,Review Text,Division Name,Department Name,label,input_ids,attention_mask,pred_Division Name,pred_prob_Division Name,pred_Department Name,pred_prob_Department Name
0,,. such a fun jacket ! great to wear in the spr...,General Petite,Intimate,"[1, 2]","[0, 4, 215, 10, 1531, 8443, 27785, 372, 7, 356...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.594270,Jackets,0.864401
1,simple and elegant,simple and elegant . i thought this shirt was ...,General Petite,Tops,"[1, 4]","[0, 41918, 8, 14878, 479, 939, 802, 42, 6399, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.659328,Tops,0.980321
2,retro and pretty,retro and pretty . this top has a bit of a ret...,General,Tops,"[0, 4]","[0, 4903, 1001, 8, 1256, 479, 42, 299, 34, 10,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.642051,Tops,0.983396
3,summer/fall wear,summer / fall wear . i first spotted this on a...,General Petite,Dresses,"[1, 1]","[0, 18581, 2089, 1589, 1136, 3568, 479, 939, 7...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.556274,Dresses,0.975182
4,perfect except slip,perfect except slip . this is my new favorite ...,General Petite,Dresses,"[1, 1]","[0, 20473, 4682, 9215, 479, 42, 16, 127, 92, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.588676,Dresses,0.984927


You can try to get your metric to see if it matches your last traing epoch's above

In [ ]:
f1_score(df_val['Division Name'],df_val['pred_Division Name'],average='macro')

0.45155638510174806

In [ ]:
f1_score(df_val['Department Name'],df_val['pred_Department Name'],average='macro')

0.6847532204330715

### Predict Test set

We will go through details on how to make a prediction on a completely new and raw dataset using our trained model. For now, let's reuse the sample csv and pretend it's our test set

In [ ]:
df_test = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig').sample(frac=0.2,random_state=1)
# drop NaN values in the label column
df_test = df_test[~df_test['Department Name'].isna()].reset_index(drop=True)

# save the label, as we will calculate some metrics later. We also filter out labels with NaN Review Text,
# as there will be a filtering processing on the test set
true_labels = df_test.loc[~df_test['Review Text'].isna(),'Department Name'].values 

# drop the label (you don't need to, but this is necessary to simulate an actual test set)
df_test.drop(['Division Name','Department Name'],axis=1,inplace=True)

In [ ]:
_test_dset = Dataset.from_pandas(df_test)
_test_dset_predicted = controller.predict_raw_dset(_test_dset,
                                                   do_filtering=True, # since we have some text filtering in the processing
                                                  )

Filter (num_proc=20):   0%|          | 0/4692 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

-------------------- Start making predictions --------------------


Map:   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4528 [00:00<?, ? examples/s]

In [ ]:
df_test_predicted = _test_dset_predicted.to_pandas()

In [ ]:
df_test_predicted.head()

,Title,Review Text,input_ids,attention_mask,pred_Division Name,pred_prob_Division Name,pred_Department Name,pred_prob_Department Name
0,perfect for work and play,perfect for work and play . this shirt works f...,"[0, 20473, 13, 173, 8, 310, 479, 42, 6399, 136...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.660698,Tops,0.986139
1,,. i don't know why i had the opposite problem ...,"[0, 4, 939, 218, 75, 216, 596, 939, 56, 5, 548...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.702822,Bottoms,0.989319
2,great pants,great pants . thes e cords are great--lightwei...,"[0, 12338, 9304, 479, 5, 29, 364, 37687, 32, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.717057,Bottoms,0.989690
3,surprisingly comfy for a button down,surprisingly comfy for a button down . i am a ...,"[0, 33258, 3137, 24382, 13, 10, 6148, 159, 479...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.682126,Tops,0.984913
4,short and small,short and small . the shirt is mostly a thick ...,"[0, 20263, 8, 650, 479, 5, 6399, 16, 2260, 10,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.597921,Tops,0.949482


Let's quickly check the f1 score to make sure everything works correctly

In [ ]:
f1_score(true_labels,df_test_predicted['pred_Department Name'],average='macro')

0.7100467903686832

Predict top k results

In [ ]:
_test_dset = Dataset.from_pandas(df_test)
_test_dset_predicted = controller.predict_raw_dset(_test_dset,
                                                   do_filtering=True,
                                                   topk=3
                                                  )

Filter (num_proc=20):   0%|          | 0/4692 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

-------------------- Start making predictions --------------------


Map:   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4528 [00:00<?, ? examples/s]

In [ ]:
df_test_predicted = _test_dset_predicted.to_pandas()

df_test_predicted.head()

,Title,Review Text,input_ids,attention_mask,pred_Division Name,pred_prob_Division Name,pred_Department Name,pred_prob_Department Name
0,perfect for work and play,perfect for work and play . this shirt works f...,"[0, 20473, 13, 173, 8, 310, 479, 42, 6399, 136...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[General, General Petite, Initmates]","[0.66069835, 0.32653692, 0.012764811]","[Tops, Intimate, Trend]","[0.9861385, 0.011745961, 0.0012178769]"
1,,. i don't know why i had the opposite problem ...,"[0, 4, 939, 218, 75, 216, 596, 939, 56, 5, 548...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[General, General Petite, Initmates]","[0.70282197, 0.2924466, 0.004731474]","[Bottoms, Intimate, Trend]","[0.98931926, 0.0058960244, 0.0033724634]"
2,great pants,great pants . thes e cords are great--lightwei...,"[0, 12338, 9304, 479, 5, 29, 364, 37687, 32, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[General, General Petite, Initmates]","[0.7170573, 0.27744323, 0.005499463]","[Bottoms, Intimate, Trend]","[0.9896898, 0.0068636425, 0.002742845]"
3,surprisingly comfy for a button down,surprisingly comfy for a button down . i am a ...,"[0, 33258, 3137, 24382, 13, 10, 6148, 159, 479...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[General, General Petite, Initmates]","[0.68212646, 0.30411962, 0.013753915]","[Tops, Intimate, Trend]","[0.9849125, 0.012397888, 0.0013534387]"
4,short and small,short and small . the shirt is mostly a thick ...,"[0, 20263, 8, 650, 479, 5, 6399, 16, 2260, 10,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[General, General Petite, Initmates]","[0.597921, 0.34346518, 0.058613785]","[Tops, Intimate, Trend]","[0.9494818, 0.048317112, 0.0011069248]"


In [ ]:
# Since we have some metadatas (Title), we need to define a dictionary containing those values
raw_content={'Review Text': 'This shirt is so comfortable I love it!',
             'Title': 'Great shirt'}

In [ ]:
controller.data_store.num_proc=1

In [ ]:
df_result = controller.predict_raw_text(raw_content,topk=3)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

-------------------- Start making predictions --------------------


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
df_result

,Review Text,Title,input_ids,attention_mask,pred_Division Name,pred_prob_Division Name,pred_Department Name,pred_prob_Department Name
0,great shirt . this shirt is so comfortable i l...,great shirt,"[0, 12338, 6399, 479, 42, 6399, 16, 98, 3473, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[General, General Petite, Initmates]","[0.58612436, 0.37989792, 0.033977736]","[Tops, Intimate, Trend]","[0.96090233, 0.03670551, 0.00122781]"
